In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('spam.csv',encoding='latin-1')
df


In [ ]:
df.shape

## 1.Data Cleaning

In [ ]:
df.info()

In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [ ]:
df.shape

In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
df.shape

In [ ]:
df.rename(columns={'v1':'target','v2':'text'},inplace=True)

In [ ]:
df.sample(7)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
encoder.fit_transform(df['target'])

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
df.duplicated()

In [ ]:
df[df.duplicated()]

In [ ]:
df=df.drop_duplicates(keep='first')

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

## 2.Exploratory Data Analysis

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(df['target'].value_counts(),labels=['ham','spam'],autopct="%0.2f")

In [ ]:
df['num_character']=df['text'].apply(len)

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df.loc[:, 'num_character'] = df['text'].apply(len)

In [ ]:
df.head()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
df['text'].apply(lambda x:nltk.word_tokenize(x))

In [ ]:
df.loc[:,'num_words']=df.loc[:,'text'].apply(lambda x:nltk.word_tokenize(x))

In [ ]:
df.head()

In [ ]:
df.drop(columns=['num_words'],inplace=True)

In [ ]:
df.head()

In [ ]:
df.loc[:,'num_words']=df.loc[:,'text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df.loc[:,'num_sentences']=df.loc[:,'text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df[['num_character','num_words','num_sentences']].describe()

In [ ]:
df['target']=encoder.fit_transform(df['target'])

In [ ]:
df.head()

In [ ]:
df[df['target']==0][['num_character','num_words','num_sentences']].describe()

In [ ]:
df[df['target']==1][['num_character','num_words','num_sentences']].describe()

## 3. Data Preprocessing 
#### . lowercase <br>  . tokenization<br>  . removing special characters<br>  . removing stop words and punctuation  <br>  . stemming

In [ ]:
def transform_text(text):
    text=text.lower()
    text=nltk.word_tokenize(text)
    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)

    text=y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text=y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)
            

In [ ]:
transform_text('hi how are travelled eating you eaten doing plaing 32%')

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
import string
string.punctuation

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
ps.stem('fucking')

In [ ]:
df['transformed_text']=df['text'].apply(transform_text)

In [ ]:
df.head()

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
wc=WordCloud(width=800,height=400,background_color='white')

In [ ]:
df.head()

In [ ]:
spam=wc.generate(df[df['target']==1]['transformed_text'].str.cat(sep=" "))

In [ ]:
plt.imshow(spam)

## 4.Model Building

In [178]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [177]:
X=cv.fit_transform(df['transformed_text']).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [179]:
X.shape

(5169, 6708)

In [180]:
y=df['target'].values
y

array([0, 0, 1, ..., 0, 0, 0])

In [181]:
from sklearn.model_selection import train_test_split

In [182]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [183]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [184]:
gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

In [185]:
gnb.fit(X_train,y_train)
y_pred1=gnb.predict(X_test)

In [186]:
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

0.8800773694390716
[[792 104]
 [ 20 118]]
0.5315315315315315


In [187]:
mnb.fit(X_train,y_train)
y_pred2=mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

0.9642166344294004
[[871  25]
 [ 12 126]]
0.8344370860927153


In [188]:
bnb.fit(X_train,y_train)
y_pred3=bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

0.9700193423597679
[[893   3]
 [ 28 110]]
0.9734513274336283


## Testing tfidf

In [189]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [190]:
X=tfidf.fit_transform(df['transformed_text']).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [191]:
y=df['target'].values
y

array([0, 0, 1, ..., 0, 0, 0])

In [192]:
X_train1,X_test1,y_train1,y_test1=train_test_split(X,y,test_size=0.2,random_state=2)

In [193]:
gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

In [194]:
gnb.fit(X_train1,y_train1)
y_pred11=gnb.predict(X_test1)

In [195]:
print(accuracy_score(y_test1,y_pred11))
print(confusion_matrix(y_test1,y_pred11))
print(precision_score(y_test1,y_pred11))

0.8762088974854932
[[793 103]
 [ 25 113]]
0.5231481481481481


In [196]:
mnb.fit(X_train1,y_train1)
y_pred22=mnb.predict(X_test1)
print(accuracy_score(y_test1,y_pred22))
print(confusion_matrix(y_test1,y_pred22))
print(precision_score(y_test1,y_pred22))

0.9593810444874274
[[896   0]
 [ 42  96]]
1.0


In [197]:
bnb.fit(X_train1,y_train1)
y_pred33=bnb.predict(X_test1)
print(accuracy_score(y_test1,y_pred33))
print(confusion_matrix(y_test1,y_pred33))
print(precision_score(y_test1,y_pred33))

0.9700193423597679
[[893   3]
 [ 28 110]]
0.9734513274336283


In [198]:
# We go with tfidf and mnb